# 0. 필요모듈 import

In [ ]:
import numpy as np
import pandas as pd
import requests # 크롤링에 사용하는 패키지
from bs4 import BeautifulSoup # html 변환에 사용함
import time
import json

# 1. 접속 가능 여부 파악하기

## 접속하려는 url 파악하기
- 동적크롤링이 필요한 페이지
- 네트워크 검사를 통해 request url, request method, referer, user-agent 파악
- 접근 가능 여부 확인하기 (request code = 200)

In [ ]:
# 카테고리 데이터 가져오기
# categoryId=0부터 categoryId=10까지
url = 'https://api.itemscout.io/api/v2/keyword/trend/?period=WEEKLY&categoryId=1'

# 헤더정보 필요
header = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'referer': 'https://itemscout.io/'
}

response = requests.get(url, headers=header)
if response.status_code == requests.codes.ok:
    print('접속성공')
response.text

접속성공


'{"status":"success","data":{"trendRankData":[{"keyword":"판초우의","change":"UP","rankedReason":"SHOPPING_POPULAR","rank":1,"keywordId":1507485,"searchCount":23280,"productCount":351124,"firstCategory":724,"competitionIntensity":15.08},{"keyword":"남자와이드팬츠","change":"NEW","rankedReason":"SHOPPING_POPULAR","rank":2,"keywordId":87071,"searchCount":10280,"productCount":2104486,"firstCategory":716,"competitionIntensity":204.72},{"keyword":"남자우비","change":"NEW","rankedReason":"NAVER_POPULAR","rank":3,"keywordId":87077,"searchCount":8200,"productCount":768609,"firstCategory":724,"competitionIntensity":93.73},{"keyword":"여성레인코트","change":"NEW","rankedReason":"NAVER_POPULAR","rank":4,"keywordId":85847,"searchCount":4510,"productCount":441455,"firstCategory":702,"competitionIntensity":97.88},{"keyword":"남자여름정장","change":"NEW","rankedReason":"SHOPPING_POPULAR","rank":5,"keywordId":81571,"searchCount":7990,"productCount":699996,"firstCategory":720,"competitionIntensity":87.61},{"keyword":"단체티주문제작","c

# 2. 일간 트렌드 정보 가져오기

## 페이지에 접속해서 필요한 데이터를 리스트에 저장하기

In [ ]:
# 일간 트렌드 전체 카테고리 가져오기

weekly_keyword = []  # keyword만 추출
weekly_all = []  # 모든 정보 추출

# 카테고리 수만큼 range값에 삽입
for i in range(11):
  url = f'https://api.itemscout.io/api/v2/keyword/trend/?period=WEEKLY&categoryId={i}'
  # url 정상적으로 가져오는 것 확인 완료

  # 헤더정보 필요
  header = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'referer': 'https://itemscout.io/'
  }

  response = requests.get(url, headers=header)
  if response.status_code == requests.codes.ok:
      print('접속성공')

  # 받아온 결과를 json 형태로 보기 쉽게 변환
  data = json.loads(response.text) # json파일을 리스트로 받아옴
  rank_list = data['data']['trendRankData']

  # # 1위부터 10위까지의 값 추출
  top20_all = []
  top20_keyword = []
  for j in range(20):
    keyword = rank_list[j]['keyword']
    # 단순히 랭킹만 알기 위한 keyword만 수집
    top20_keyword.append(keyword)

    change = rank_list[j]['change']
    # rankedReason = rank_list[j]['rankedReason']
    searchCount = rank_list[j]['searchCount']
    competitionIntensity = rank_list[j]['competitionIntensity']

    # 다양한 정보를 포함하는 dict생성
    # trend_data = {'keyword': keyword, 'change': change, 'rankedReason':rankedReason, 'searchCount' : searchCount, 'competitionIntensity':competitionIntensity}
    trend_data = {'keyword': keyword, 'change': change, 'searchCount' : searchCount, 'competitionIntensity':competitionIntensity}
    top20_all.append(trend_data)

  weekly_keyword.append(top20_keyword)
  weekly_all.append(top20_all)

접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공


In [ ]:
# 아래의 명령어로 결과값을 확인할 수 있다.
weekly_keyword, weekly_all

## 모든 데이터를 담은 리스트를 데이터프레임으로

In [ ]:
# 각 카테고리에 대한 데이터프레임을 담을 리스트
dfs = []

# 반복문을 통해 데이터프레임 생성 및 리스트에 추가
for i in weekly_all:
    df_category = pd.DataFrame(i)
    df_category['change'] = df_category['change'].map({'NEW': '▲', 'UP': '▲', 'DOWN': '▼', 'NO_CHANGE': '-'})
    dfs.append(df_category)

In [ ]:
# 첫 번째 데이터프레임을 기준으로 시작
result_df = dfs[0]

# 나머지 데이터프레임을 반복문을 통해 오른쪽으로 합침
for df in dfs[1:]:
    result_df = pd.concat([result_df, df], axis=1)

result_df

,keyword,change,searchCount,competitionIntensity,keyword,change,searchCount,competitionIntensity,keyword,change,...,searchCount,competitionIntensity,keyword,change,searchCount,competitionIntensity,keyword,change,searchCount,competitionIntensity
0,개업축하화분,▲,6810,37.72,판초우의,▲,23280,15.08,레인부츠,▲,...,18790,1.72,개업축하화분,▲,6810,37.72,가평빠지,▼,175900,0.01
1,과일바구니,▲,20530,18.89,남자와이드팬츠,▲,10280,204.72,14k목걸이,▲,...,35770,4.98,설거지,▼,54450,17.17,화환,-,58800,23.52
2,닭발,▲,69810,1.02,남자우비,▲,8200,93.73,헌터레인부츠,▲,...,30750,15.23,도장,▲,67300,24.68,기차여행,▼,39780,0.95
3,신일선풍기,▲,266300,0.33,여성레인코트,▲,4510,97.88,크록스,▼,...,125900,13.15,모기기피제,▲,88800,1.14,개업화분,▼,53800,7.54
4,중고폰,▲,66700,1.97,남자여름정장,▲,7990,87.61,여성장화,▲,...,11450,1.96,사다리,▲,117300,13.63,수영장호텔,▼,15950,6.85
5,도장,▲,67300,24.68,단체티주문제작,▲,14750,2.48,여행용파우치,▲,...,30650,62.72,수건,▲,64700,62.43,꽃바구니,▲,15890,23.38
6,레인부츠,▲,139200,10.40,파란색티셔츠,▼,4070,7.34,wpc양산,▼,...,8250,34.85,후라이팬,▲,29950,103.98,에어컨청소,▼,282700,0.97
7,장어,▲,103900,1.65,아미반팔티,▲,44300,1.25,남자장화,▲,...,181300,3.29,텀블러,▲,178100,18.84,소노벨경주오션플레이,▼,2090,0.30
8,화환,▲,58800,23.52,단체티,▲,29400,14.83,14k귀걸이,▲,...,61400,13.95,강아지쿨매트,▲,51730,2.91,문화상품권,▼,137600,0.02
9,설거지,▼,54450,17.17,폴로린넨셔츠,▲,36910,0.90,장우산,▲,...,6060,25.13,짱구,▼,110600,6.84,선불유심,▼,13890,0.16


## 주간 트렌드 키워드만 담은 리스트를 데이터프레임으로

In [ ]:
df = pd.DataFrame(data = weekly_keyword).T
df.columns=['전체', '패션의류','패션잡화','화장품/미용','디지털/가전','가구/인테리어','출산/육아','식품','스포츠/레저','생활/건강', '여가/편의']
df

,전체,패션의류,패션잡화,화장품/미용,디지털/가전,가구/인테리어,출산/육아,식품,스포츠/레저,생활/건강,여가/편의
0,개업축하화분,판초우의,레인부츠,미백크림,서큘레이터,세라믹식탁,백일상대여,닭갈비,자이로볼,개업축하화분,가평빠지
1,과일바구니,남자와이드팬츠,14k목걸이,비타민c앰플,신일선풍기,침대프레임,아기수영복,과일바구니,스노쿨링마스크,설거지,화환
2,닭발,남자우비,헌터레인부츠,탈모샴푸,중고폰,접이식매트리스,답례품,닭발,농구공,도장,기차여행
3,신일선풍기,여성레인코트,크록스,타투스티커,모기퇴치기,거실테이블,순금돌반지,장어,래쉬가드,모기기피제,개업화분
4,중고폰,남자여름정장,여성장화,바닐라코파우더,선풍기,여름이불,아동래쉬가드,복숭아,족구화,사다리,수영장호텔
5,도장,단체티주문제작,여행용파우치,일회용샴푸,이동식에어컨,리클라이너쇼파,탯줄도장,그릭요거트,오토바이헬멧,수건,꽃바구니
6,레인부츠,파란색티셔츠,wpc양산,ahc선스틱,보조배터리,토퍼매트리스,워터밤물총,매실장아찌,요가양말,후라이팬,에어컨청소
7,장어,아미반팔티,남자장화,오브제,음식물처리기,붙박이장,어린이장화,참외,내셔널지오그래픽,텀블러,소노벨경주오션플레이
8,화환,단체티,14k귀걸이,아모스녹차실감샴푸,벽걸이에어컨,게이밍의자,어린이집답례품,비타민,골프장갑,강아지쿨매트,문화상품권
9,설거지,폴로린넨셔츠,장우산,엘리자베스아덴립밤,손선풍기,목베개,아동레인부츠,마늘,골프비옷,짱구,선불유심


# 결과 저장하기

In [ ]:
# csv 파일로 저장
# 일간트렌드 데이터마다 파일명 다르게 저장
from datetime import datetime
import pytz

# 현재 시간을 가져옵니다.
current_time = datetime.now()

# UTC 시간대로 변경합니다.
utc_time = current_time.replace(tzinfo=pytz.utc)

# 한국 시간대로 변경합니다.
korea_time = utc_time.astimezone(pytz.timezone('Asia/Seoul'))

today_date = korea_time.strftime('%y년 %m월 %W주차')

In [ ]:
today_date

'24년 06월 26주차'

In [ ]:
writer = pd.ExcelWriter(f'/content/drive/My Drive/trend_tracking/weekly_trend_{today_date}.xlsx')
df.to_excel(writer, sheet_name="keyword")
result_df.to_excel(writer, sheet_name="all_datas")
writer.close()